## Python __new__ 和 __init__的区别

__new__和__init__参数的不同
__new__所接收的第一个参数是cls，而__init__所接收的第一个参数是self。这是因为当我们调用__new__的时候，该类的实例还并不存在（也就是self所引用的对象还不存在），所以需要接收一个类作为参数，从而产生一个实例。而当我们调用__init__的时候，实例已经存在，因此__init__接受self作为第一个参数并对该实例进行必要的初始化操作。这也意味着__init__是在__new__之后被调用的。

关于 Python 新式类和老式类在这篇文章不做过多讨论，因为老式类是 Python2 中的概念，现在基本没人再会去用老式类，新式类必须显示地继承 object，而 Python3 中，只有新式类，默认继承了 object，无需显示指定，本文代码都是基于 Python3 来讨论。

### __init__方法
__init__方法负责对象的初始化，系统执行该方法前，其实该对象已经存在了，要不然初始化什么东西呢？先看例子：

In [17]:
# class A(object): python2 必须显示地继承object

class A1(object):
    def __init__(self):
        print("__init__ ")

    def __new__(cls):
        print("__new__ ")
        return super(A1, cls).__new__(cls)

    def __call__(self):  # 可以定义任意参数
        print('__call__ ')

A1()

__new__ 
__init__ 


从输出结果来看， __new__方法先被调用，返回一个实例对象，接着 __init__ 被调用。 
__call__方法并没有被调用，这个我们放到最后说，先来说说前面两个方法，稍微改写成：

In [18]:
class A2(object):
    def __init__(self):
        print("__init__ ")
        print(self)

    def __new__(cls):
        print("__new__ ")
        self = super(A2, cls).__new__(cls)
        print(self)
        return self
    
A2()

__new__ 
__init__ 


从输出结果来看，__new__ 方法的返回值就是类的实例对象，这个实例对象会传递给 __init__ 方法中定义的 self 参数，以便实例对象可以被正确地初始化。

如果 __new__ 方法不返回值（或者说返回 None）那么 __init__ 将不会得到调用，这个也说得通，因为实例对象都没创建出来，调用 init 也没什么意义，此外，Python 还规定，__init__ 只能返回 None 值，否则报错。

下面是zhihu-oauth的代码案例：


In [14]:
class _Anonymous(object):
    def __init__(self):
        self.id = 0
        self.name = '匿名用户'

    def __getattr__(self, _):
        # 匿名用户除了姓名和 ID 以外所有属性均为 None
        return None


ANONYMOUS = _Anonymous()
"""
.. role:: py_code(code)
   :language: python
统一的匿名用户对象，可以使用 :py_code:`if people is ANONYMOUS:` 判断是否是匿名用户
"""


class People(object):
    def __new__(cls, pid, cache, session):
        if pid == '0':
            return ANONYMOUS
        else:
            return super(People, cls).__new__(cls)

    def __init__(self, pid, cache, session):
        self._over_e = None
        super(People, self).__init__(pid, cache, session)

### __call__ 方法
关于 __call__ 方法，不得不先提到一个概念，就是可调用对象（callable），我们平时自定义的函数、内置函数和类都属于可调用对象，但凡是可以把一对括号()应用到某个对象身上都可称之为可调用对象，判断对象是否为可调用对象可以用函数 callable

如果在类中实现了 __call__ 方法，那么实例对象也将成为一个可调用对象，我们回到最开始的那个例子：

In [19]:
a = A1()
print(callable(a))

__new__ 
__init__ 
True


In [20]:
a()

__call__ 


下面我们来看一个案例

In [32]:
class Entity(object):
    """调用实体来改变实体的位置。"""

    def __init__(self, size, x, y):
        self.x, self.y = x, y
        self.size = size

    def __call__(self, x, y):
        '''改变实体的位置'''
        self.x, self.y = x, y
    
e = Entity(100,1,2) 
print(e.x,e.y)
e(2,3)
print(e.x,e.y)

(1, 2)
(2, 3)
